In [1]:
!pip install pylibjpeg --find-links /kaggle/input/frozen-packages-for-offline-use/frozen_packages --no-index
!pip install python-gdcm --find-links /kaggle/input/frozen-packages-for-offline-use/frozen_packages --no-index
!pip install pylibjpeg-libjpeg --find-links /kaggle/input/frozen-packages-for-offline-use/frozen_packages --no-index

Looking in links: /kaggle/input/frozen-packages-for-offline-use/frozen_packages
Processing /kaggle/input/frozen-packages-for-offline-use/frozen_packages/pylibjpeg-1.4.0-py3-none-any.whl
Looking in links: /kaggle/input/frozen-packages-for-offline-use/frozen_packages
Processing /kaggle/input/frozen-packages-for-offline-use/frozen_packages/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: /kaggle/input/frozen-packages-for-offline-use/frozen_packages
Processing /kaggle/input/frozen-packages-for-offline-use/frozen_packages/pylibjpeg_libjpeg-1.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom
import os
import nibabel as nib
import SimpleITK as sitk
from tqdm import tqdm
from tensorflow import keras
from keras import layers, models
import tensorflow_io as tfio
import cv2
import warnings
warnings.filterwarnings('ignore')
from skimage.transform import resize
from tqdm import tqdm
import tensorflow_addons as tfa

In [3]:
def load_image(path):
    img = pydicom.read_file(path)
    img = img.pixel_array
    return cv2.resize(img[35:-35,35:-35], (160,160))

In [4]:
def window(img):
    img[np.where(img>1900)]=1900
    img[np.where(img<0)]=0
    return img/1900.0

In [5]:
def channeling(img):
    
    return np.stack([img, img, img], axis=-1)

In [28]:
def extract(path):
    final_ct = load_image(path+"/"+"10.dcm").reshape(1,160,160) 
    for image in range(11, len(os.listdir(path))-10):
        img = load_image(path+"/"+str(image)+".dcm").reshape(1,160,160)
        final_ct = np.append(final_ct, img, axis=0)
        
    resized_final_ct = cv2.resize(final_ct[:,:,0], (160,160)).reshape(1,160,160)
    for i in range(1,final_ct.shape[1]):
        resized_final_ct = np.append(resized_final_ct, cv2.resize(final_ct[:,:,i], (160,160)).reshape(1,160,160), axis=0)
  
    resized_final_ct = np.transpose(resized_final_ct,axes=[1,0,2])
    return window(channeling(resized_final_ct))

In [7]:
model = tf.keras.models.load_model('../input/rsna-image-classification/final_model/model')

2022-10-12 16:34:10.030377: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
inputs = tf.keras.layers.Input(shape=(160, 160, 3),name= "input_2")
x = model.get_layer("input_1")(inputs)
x = model.get_layer("densenet201")(x)
class_model = tf.keras.Model(inputs=inputs, outputs=x)


In [9]:
input1 = keras.Input((100, 1920))

s3 = keras.layers.Bidirectional(keras.layers.LSTM(1920, return_sequences=True))(input1)
s3 = keras.layers.Bidirectional(keras.layers.LSTM(1920, return_sequences=False))(s3)
s3 = keras.layers.Dense(512, activation='relu')(s3)
s3 = keras.layers.BatchNormalization()(s3)
s3 = keras.layers.Dense(128, activation='relu')(s3)
s3 = keras.layers.BatchNormalization()(s3)
s3 = keras.layers.Dense(32, activation='relu')(s3)

outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(s3)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(s3)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(s3)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(s3)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(s3)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(s3)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(s3)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(s3)
                                               
s_model = keras.models.Model(inputs=input1, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7])

In [10]:
s_model.load_weights("../input/s-model-v1/final_model/model")

In [29]:
patient_pred = {}
UIDs = os.listdir("../input/rsna-2022-cervical-spine-fracture-detection/test_images")
for p in UIDs:
    path = "../input/rsna-2022-cervical-spine-fracture-detection/test_images/"+p+"/"
    ct = extract(path)
    ct = class_model.predict(ct)[30:130]
    ct = s_model.predict(ct.reshape(1,100,1920))
    for i in range(1, 9):
        if i==8:
            patient_pred[p+"_"+"patient_overall"] = ct[0][0][0]
            break
        else:
            patient_pred[p+"_"+"C"+str(i)] = ct[i][0][0]

In [30]:
submission  = pd.DataFrame.from_dict({"row_id": patient_pred.keys(), "fractured": patient_pred.values()})
submission.to_csv('./submission.csv', index=False)
submission

,row_id,fractured
0,1.2.826.0.1.3680043.22327_C1,0.059678
1,1.2.826.0.1.3680043.22327_C2,0.198587
2,1.2.826.0.1.3680043.22327_C3,0.133383
3,1.2.826.0.1.3680043.22327_C4,0.253855
4,1.2.826.0.1.3680043.22327_C5,0.082835
5,1.2.826.0.1.3680043.22327_C6,0.353808
6,1.2.826.0.1.3680043.22327_C7,0.069337
7,1.2.826.0.1.3680043.22327_patient_overall,0.472664
8,1.2.826.0.1.3680043.25399_C1,0.065543
9,1.2.826.0.1.3680043.25399_C2,0.219612
